# H4221 - Rapport Machine Learning

[a compléter]

Auteurs: 
- DUBILLOT Elise
- FLANDRE Corentin
- THOMAS Colin

## Introduction

L'objectif du projet est de tester différents modèles de [a compléter]

In [13]:
print("hello-world")

## Chargement du jeu de données

In [14]:
import pandas as pd

df = pd.read_csv("kc_house_data.csv")
print(df)


### Traitement du jeu de données

[blabla à compléter]

Un traitement sur la date est à réaliser. Nous faisons le choix d'identifier uniquement le mois de la vente pour l'utilisr plus tard. Le jeu de données prend en compte des dates entre Mai 2014 et Mai 2015

In [18]:
# print(df["date"])
print(f"size: {len(df)}")
print(df["date"])
for i in range(len(df)):
    year = int(df["date"][i][0:4])
    month = int(df["date"][i][4:6])
    # day = int(df["date"][i][6:8]) # donnee non utilise pour notre traitement
    new_date = "error"
    if year==2014:
        if month==5:
            new_date = "month1"
        elif month==6:
            new_date = "month2"    
        elif month==7:
            new_date = "month3" 
        elif month==8:
            new_date = "month4" 
        elif month==9:
            new_date = "month5" 
        elif month==10:
            new_date = "month6" 
        elif month==11:
            new_date = "month7"
        elif month==12:
            new_date = "month8"
    elif year==2015:
        if month==1:
            new_date = "month9"
        elif month==2:
            new_date = "month10"    
        elif month==3:
            new_date = "month11" 
        elif month==4:
            new_date = "month12" 
        elif month==5:
            new_date = "month13" 
    df["date"][i] = new_date
print("--------------------")
# print(df["date"])
print(df)
    
    


size: 21613
0         month6
1         month8
2        month10
3         month8
4        month10
          ...   
21608     month1
21609    month10
21610     month2
21611     month9
21612     month6
Name: date, Length: 21613, dtype: object


ValueError: invalid literal for int() with base 10: 'mont'

## Exploration du jeu de données

plt.xlabel("Taille du terrain en sqrft")
plt.ylabel('Prix moyen (en $)')
plt.title('Prix en fonction de la taille du terrain')
plt.show()

dfsorted = df[["price", "zipcode"]]
dfsorted.sort_values(by=["zipcode"], inplace=True)

dfpbyb = dfsorted.groupby("zipcode")["price"].mean()

dfpbyb.to_numpy()
plt.plot(dfpbyb[:])

plt.xlabel("ZIPCODE")
plt.ylabel('Prix moyen (en $)')
plt.title('Prix en fonction du zipcode')
plt.show()


dfsorted = df[["price", "lat"]]
dfsorted.sort_values(by=["lat"], inplace=True)

dfpbyb = dfsorted.groupby("lat")["price"].mean()

dfpbyb.to_numpy()
plt.hist(df["lat"], bins = 30)

plt.xlabel("Latitude")
plt.title('Répartition des données sur la latitude')
plt.show()

dfsorted = df[["price", "long"]]
dfsorted.sort_values(by=["long"], inplace=True)

dfpbyb = dfsorted.groupby("long")["price"].mean()

dfpbyb.to_numpy()
plt.hist(df["long"])

plt.xlabel("Longitude")
plt.title('Répartition des données sur la longitude')
plt.show()


In [ ]:
import plotly.express as px

fig = px.density_mapbox(df, lat='lat', lon='long', z=1,
                        mapbox_style="stamen-terrain")
 
fig

On remarque ici plusieurs cas aberrants, que l'on traitera dans la partie traitement des données.
Par exemple, dans le cas du nombre de chambre, on remarque une donnée qui représente une maison de 33 chambres, coûtant moins qu'une maison moyenne de 7 chambres et n'ayant que 1.25 salles de bain.
On remarque également que le prix et la taille du terrain ne semblent pas avoir un lien au delà de environ 0.05 sqft. On peut supposer que cela est du à la multiplicité de facteurs influençant le prix.
